In [1]:
# 导入函数库
from jqdatasdk import *
import pandas as pd 
import seaborn as sns
#显示行数
pd.set_option('display.max_rows',20)

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改


In [2]:
# auth('ID','Password')

auth('138***','****') #修改为自己的用户名密码

auth success 


In [3]:
get_query_count()

{'total': 1000000, 'spare': 983374}

In [4]:
get_all_securities()

,display_name,name,start_date,end_date,type
000001.XSHE,平安银行,PAYX,1991-04-03,2200-01-01,stock
000002.XSHE,万科A,WKA,1991-01-29,2200-01-01,stock
000004.XSHE,国农科技,GNKJ,1990-12-01,2200-01-01,stock
000005.XSHE,世纪星源,SJXY,1990-12-10,2200-01-01,stock
000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01,stock
...,...,...,...,...,...
688596.XSHG,正帆科技,ZFKJ,2020-08-20,2200-01-01,stock
688598.XSHG,金博股份,JBGF,2020-05-18,2200-01-01,stock
688599.XSHG,天合光能,THGN,2020-06-10,2200-01-01,stock
688600.XSHG,皖仪科技,WYKJ,2020-07-03,2200-01-01,stock


## 价格数据

`get_price(security, start_date=None, end_date=None, frequency='daily', fields=None, skip_paused=False, fq='pre', count=None, panel=True, fill_paused=True)`

**参数**

- security: 一支股票代码或者一个股票代码的list
- count: **与 start_date 二选一，不可同时使用**. 数量, 返回的结果集的行数, 即表示获取 end_date 之前几个 frequency 的数据
- start_date: **与 count 二选一，不可同时使用**. 字符串或者 [datetime.datetime]/[datetime.date] 对象, 开始时间.
  - 如果 count 和 start_date 参数都没有, 则 start_date 生效, 值是 '2015-01-01'. 注意:
  - 当取分钟数据时, 时间可以精确到分钟, 比如: 传入 `datetime.datetime(2015, 1, 1, 10, 0, 0)` 或者 `'2015-01-01 10:00:00'`.
  - 当取分钟数据时, 如果只传入日期, 则日内时间是当日的 00:00:00.
  - 当取天数据时, 传入的日内时间会被忽略
- end_date: 格式同上, 结束时间, 默认是'2015-12-31', 包含此日期. **注意: 当取分钟数据时, 如果 end_date 只有日期, 则日内时间等同于 00:00:00, 所以返回的数据是不包括 end_date 这一天的**.
- frequency: 单位时间长度, 几天或者几分钟, 现在支持'Xd','Xm', 'daily'(等同于'1d'), 'minute'(等同于'1m'), X是一个正整数, 分别表示X天和X分钟(不论是按天还是按分钟回测都能拿到这两种单位的数据), 注意, 当X > 1时, fields只支持['open', 'close', 'high', 'low', 'volume', 'money']这几个标准字段. 默认值是daily
- fields: 字符串list, 选择要获取的行情数据字段, 默认是None(表示['open', 'close', 'high', 'low', 'volume', 'money']这几个标准字段), 支持[SecurityUnitData](https://www.joinquant.com/help/api/help?name=api#SecurityUnitData)里面的所有基本属性,，包含：['open', 'close', 'low', 'high', 'volume', 'money', 'factor', 'high_limit','low_limit', 'avg', 'pre_close', 'paused', 'open_interest'], open_interest为**期货持仓量**
- skip_paused: 是否跳过不交易日期(包括停牌, 未上市或者退市后的日期). 如果不跳过, 停牌时会使用停牌前的数据填充(具体请看SecurityUnitData的paused属性), 上市前或者退市后数据都为 nan, 但要注意:
  - 默认为 False
  - 当 skip_paused 是 True 时, 获取多个标的时需要将panel参数设置为False(panel结构需要索引对齐)
- fq: 复权选项:
  - `'pre'`: 前复权
  - `None`: 不复权, 返回实际价格
  - `'post'`: 后复权
- panel：当传入股票列表时，指定返回结果是否使用panel格式，默认为True；指定panel=False时返回dataframe格式。
- fill_paused：对于停牌股票的价格处理，默认为True；True表示用pre_close价格填充；False 表示使用NAN填充停牌的股票价格。

**返回**

- **请注意, 为了方便比较一只股票的多个属性, 同时也满足对比多只股票的一个属性的需求, 我们在security参数是一只股票和多只股票时返回的结构完全不一样 (默认panel=True时)**

- 如果是一支股票, 则返回[pandas.DataFrame]对象, 行索引是[datetime.datetime]对象, 列索引是行情字段名字, 比如'open'/'close'. 比如: `get_price('000300.XSHG')[:2]` 返回:

  | ---                 | open    | close   | high    | low     | volume      | money          |
  | :------------------ | :------ | :------ | :------ | :------ | :---------- | :------------- |
  | 2015-01-05 00:00:00 | 3566.09 | 3641.54 | 3669.04 | 3551.51 | 451198098.0 | 519849817448.0 |
  | 2015-01-06 00:00:00 | 3608.43 | 3641.06 | 3683.23 | 3587.23 | 420962185.0 | 498529588258.0 |

- 如果是多支股票, 则返回[pandas.Panel]对象, 里面是很多[pandas.DataFrame]对象, 索引是行情字段(open/close/…), 每个[pandas.DataFrame]的行索引是[datetime.datetime]对象, 列索引是股票代号. 比如`get_price(['000300.XSHG', '000001.XSHE'])['open'][:2]`返回:

  | ---                 | 000300.XSHG | 000001.XSHE |
  | :------------------ | :---------- | :---------- |
  | 2015-01-05 00:00:00 | 3566.09     | 13.21       |
  | 2015-01-06 00:00:00 | 3608.43     | 13.09       |

In [5]:
df = get_price('000001.XSHE')
df

,open,close,high,low,volume,money
2015-01-05,10.24,10.26,10.43,9.99,446489383.0,4.565388e+09
2015-01-06,10.15,10.11,10.50,9.96,338159640.0,3.453446e+09
2015-01-07,9.97,9.92,10.14,9.80,265374121.0,2.634796e+09
2015-01-08,9.93,9.58,9.97,9.55,219732012.0,2.128003e+09
2015-01-09,9.55,9.66,10.17,9.42,391555169.0,3.835378e+09
...,...,...,...,...,...,...
2015-12-25,9.60,9.62,9.67,9.56,51558377.0,4.956380e+08
2015-12-28,9.64,9.29,9.66,9.29,106046218.0,1.003010e+09
2015-12-29,9.30,9.38,9.38,9.28,79920944.0,7.447184e+08
2015-12-30,9.38,9.38,9.39,9.27,68685228.0,6.412772e+08


In [6]:
df = get_price('000001.XSHE', start_date='2020-01-01',             \
            end_date='2020-01-31 23:00:00',                         \
               frequency='daily', fields=['open', 'close'])
df

,open,close
2020-01-02,16.37,16.58
2020-01-03,16.65,16.89
2020-01-06,16.72,16.78
2020-01-07,16.84,16.86
2020-01-08,16.71,16.38
2020-01-09,16.53,16.51
2020-01-10,16.51,16.41
2020-01-13,16.47,16.70
2020-01-14,16.70,16.48
2020-01-15,16.51,16.24


In [7]:
# 获取获得000001.XSHG在2015年01月31日前2个交易日的数据
df = get_price('000001.XSHE', count = 2, end_date='2015-01-31', frequency='daily', fields=['open', 'close']) 
df

,open,close
2015-01-29,8.85,8.91
2015-01-30,8.92,8.92


In [8]:
# 获取多只股票
panel =  get_price(get_index_stocks('000903.XSHG')) # 获取中证100的所有成分股的2015年的天数据, 返回一个[pandas.Panel]
panel

,time,code,open,close,high,low,volume,money
0,2015-01-05,601857.XSHG,9.70,10.58,10.58,9.58,419005913.0,4.228175e+09
1,2015-01-06,601857.XSHG,10.76,10.40,10.97,10.23,437608197.0,4.626943e+09
2,2015-01-07,601857.XSHG,10.23,11.02,11.05,10.17,370177860.0,3.900464e+09
3,2015-01-08,601857.XSHG,10.85,10.86,11.69,10.74,478644195.0,5.362077e+09
4,2015-01-09,601857.XSHG,10.74,10.74,11.30,10.53,334132540.0,3.640000e+09
...,...,...,...,...,...,...,...,...
24395,2015-12-25,601186.XSHG,12.54,12.64,12.67,12.41,20541178.0,2.579197e+08
24396,2015-12-28,601186.XSHG,12.71,12.23,12.71,12.20,36161860.0,4.518754e+08
24397,2015-12-29,601186.XSHG,12.25,12.28,12.31,12.18,20947996.0,2.561569e+08
24398,2015-12-30,601186.XSHG,12.36,12.34,12.50,12.27,21991434.0,2.717333e+08


## 获取财务数据

`get_fundamentals(query_object, date=None, statDate=None)`

查询财务数据，详细的数据字段描述请点击[财务数据文档](https://www.joinquant.com/help/api/help?name=Stock#财务数据列表)查看

date和statDate参数只能传入一个:

- 传入date时, 查询**指定日期date收盘后所能看到的最近(对市值表来说, 最近一天, 对其他表来说, 最近一个季度)的数据**, 我们会查找上市公司在这个日期之前(包括此日期)发布的数据, 不会有未来函数.
- 传入statDate时, 查询 **statDate 指定的季度或者年份的财务数据**. 注意:

1. 由于公司发布财报不及时, 一般是看不到当季度或年份的财务报表的, 回测中使用这个数据可能会有未来函数, 请注意规避.
2. 由于估值表每天更新, 当按季度或者年份查询时, 返回季度或者年份最后一天的数据
3. 由于“资产负债数据”这个表是存量性质的， 查询年度数据是返回第四季度的数据。
4. 银行业、券商、保险专项数据只有年报数据，需传入statDate参数，当传入 date 参数 或 statDate 传入季度时返回空，请自行避免未来函数。

当 date 和 statDate 都不传入时, 相当于使用 date 参数, date 的默认值下面会描述.

**参数**

- query_object: 一个sqlalchemy.orm.query.Query对象，可以通过全局的 query 函数获取 Query 对象，[query简易教程](https://www.joinquant.com/view/community/detail/16411)
- date: 查询日期, 一个字符串(格式类似'2015-10-15')或者[datetime.date]/[datetime.datetime]对象, 为None时返回昨天（如果昨天非交易日，也会返回昨天的，不是上一个交易日）的数据；
- statDate: 财报统计的季度或者年份, 一个字符串, 有两种格式:

1. 季度: 格式是: **年 + 'q' + 季度序号**, 例如: '2015q1', '2013q4'.
2. 年份: 格式就是年份的数字, 例如: '2015', '2016'.

**返回** 返回一个 [pandas.DataFrame], 每一行对应数据库返回的每一行(可能是几个表的联合查询结果的一行), 列索引是你查询的所有字段 注意：

1. **为了防止返回数据量过大, 我们每次最多返回10000行**
2. 当相关股票上市前、退市后，财务数据返回各字段为空

In [9]:
q = query(
    valuation
).filter(
    valuation.code == '000001.XSHE'
)
df = get_fundamentals(q, '2015-10-15')
df

,id,code,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,day,pe_ratio_lyr
0,5024884,000001.XSHE,7.4984,0.4116,1.0593,1.8748,1.0246,1430867.625,1598.2791,1180405.5,1318.5129,2015-10-15,8.0713


In [10]:
# 查询平安银行2014年的年报
q = query(
        income.statDate,
        income.code,
        income.basic_eps,
    ).filter(
        income.code == '000001.XSHE',
    )

ret = get_fundamentals(q, statDate='2014')
ret

,statDate,code,basic_eps
0,2014-12-31,000001.XSHE,1.73


## 股票收益率

股票周期简单收益率： $ R_t=\frac{P_t-P_{t-1}}{P_{t-1}} $ \
对数形式：   $ r_t=\ln(1+R_t)=\ln(\frac{P_t}{P_{t-1}}) $

In [11]:
df = get_price('000001.XSHE', start_date='2020-09-01', end_date='2020-09-30 23:00:00',  \
               frequency='daily', fields=['close'])
Rt =df.pct_change().dropna()
Rt

,close
2020-09-02,0.011889
2020-09-03,-0.027415
2020-09-04,0.004027
2020-09-07,-0.001337
2020-09-08,0.032798
...,...
2020-09-24,-0.032630
2020-09-25,0.004630
2020-09-28,0.007900
2020-09-29,-0.033312
